In [2]:
#import dependancieies 
import datetime
import backtrader as bt
import pandas as pd
import math
import matplotlib



In [3]:

def start_test():
    startdate=datetime.datetime(2019, 1, 1)
    enddate=datetime.datetime(2021, 6, 1)
    tickers = ["GME","ADBE","SONY"]
    for ticker in tickers:
        run_test(ticker,startdate,enddate)

#creating the parameters for cerebro to feed data to adjust dates to your own buy date

def run_test(ticker,startdate,enddate):

    cerebro = bt.Cerebro()
    cerebro.addstrategy(Cross)
    cerebro.addwriter(bt.WriterFile, csv=True,out=f"{ticker}_trade_log.csv", rounding=2)
    cerebro.broker.set_cash(1000)

    feed = bt.feeds.YahooFinanceData(
        dataname=ticker,
        timeframe=bt.TimeFrame.Days,
        fromdate=startdate,
        todate=enddate,
        reverse=False,
    )
    # print(feed)
    cerebro.adddata(feed)
    print("starting portfolio value: %.2f" % cerebro.broker.getvalue())
    cerebro.run(stdstats=True)
    

    print("final portfolio value: %.2f" % cerebro.broker.getvalue())
    
    cerebro.plot(style='candle',)

#create strategy held object


class Cross(bt.Strategy):

    # set parameters to define fast and slow
    params = (
        ("fast", 6),
        ("slow", 13),
        ("order_percentage", 0.99),
        ("ticker", "stock"),
    )

    # define constractors
    def __init__(self):
        print("position size:", self.position.size)
        _name='Custom Cross Strategy'
        self.fast_moving_average = bt.ind.EMA(
            self.data.open, period=self.params.fast, plotname="fast moving average"
        )

        self.slow_moving_average = bt.ind.EMA(
            self.data.close, period=self.params.slow, plotname="slow moving average"
        )

        self.crossover = bt.ind.CrossOver(
            self.fast_moving_average, self.slow_moving_average
        )

    def log(self, txt, dt=None):
        """ Logging function for this strategy"""
        dt = dt or self.data.datetime[0]
        if isinstance(dt, float):
            dt = bt.num2date(dt)
        print("%s, %s" % (dt.date(), txt))

    def notify_order(self, order):
        """ Triggered upon changes to orders. """

        # Suppress notification if it is just a submitted order.
        if order.status == order.Submitted:
            return

        # Print out the date, security name, order number and status.
        dt, dn = self.datetime.date(), order.data._name
        type = "Buy" if order.isbuy() else "Sell"
        self.log(
            f"{order.data._name:<6} Order: {order.ref:3d}\tType: {type:<5}\tStatus"
            f" {order.getstatusname():<8} \t"
            f"Size: {order.created.size:9.4f} Price: {order.created.price:9.4f} "
            f"Position: {self.getposition(order.data).size}"
        )
        if order.status == order.Margin:
            return

        # Check if an order has been completed
        if order.status in [order.Completed]:
            self.log(
                f"{order.data._name:<6} {('BUY' if order.isbuy() else 'SELL'):<5} "
                # f"EXECUTED for: {dn} "
                f"Price: {order.executed.price:6.2f} "
                f"Cost: {order.executed.value:6.2f} "
                f"Comm: {order.executed.comm:4.2f} "
                f"Size: {order.created.size:9.4f} "
            )

    def notify_trade(self, trade):
        """Provides notification of closed trades."""
        if trade.isclosed:
            self.log(
                "{} Closed: PnL Gross {}, Net {},".format(
                    trade.data._name,
                    round(trade.pnl, 2),
                    round(trade.pnlcomm, 1),
                )
            )

    def next(self):
        if self.position.size == 0:
            if self.crossover > 0:
                amount_to_invest = self.params.order_percentage * self.broker.cash
                self.size = math.floor(amount_to_invest / self.data.close)

                self.log(
                    "Buy {} shares of {} at {}".format(
                        self.size,
                        self.params.ticker,
                        self.data.close[0],
                    )
                )
                self.buy(size=self.size)

        if self.position.size > 0:
            if self.crossover < 0:
                self.log(
                    "Sell {} shares of {} at {}".format(
                        self.size, self.params.ticker, self.data.close[0],
                    )
                )
                self.sell(size=self.size)


if __name__ == "__main__":
    start_test()



starting portfolio value: 1000.00
position size: 0
2019-03-04, Buy 88 shares of stock at 11.19
2019-03-05, GME    Order:   1	Type: Buy  	Status Accepted 	Size:   88.0000 Price:   11.1900 Position: 0
2019-03-05, GME    Order:   1	Type: Buy  	Status Margin   	Size:   88.0000 Price:   11.1900 Position: 0
2019-03-14, Buy 90 shares of stock at 10.95
2019-03-15, GME    Order:   2	Type: Buy  	Status Accepted 	Size:   90.0000 Price:   10.9500 Position: 90
2019-03-15, GME    Order:   2	Type: Buy  	Status Completed 	Size:   90.0000 Price:   10.9500 Position: 90
2019-03-15, GME    BUY   Price:  10.99 Cost: 989.10 Comm: 0.00 Size:   90.0000 
2019-03-15, Sell 90 shares of stock at 11.06
2019-03-18, GME    Order:   3	Type: Sell 	Status Accepted 	Size:  -90.0000 Price:   11.0600 Position: 0
2019-03-18, GME    Order:   3	Type: Sell 	Status Completed 	Size:  -90.0000 Price:   11.0600 Position: 0
2019-03-18, GME    SELL  Price:  11.09 Cost: 989.10 Comm: 0.00 Size:  -90.0000 
2019-03-18, GME Closed: PnL 

<IPython.core.display.Javascript object>

starting portfolio value: 1000.00
position size: 0
2019-03-13, Buy 3 shares of stock at 264.38
2019-03-14, ADBE   Order:  55	Type: Buy  	Status Accepted 	Size:    3.0000 Price:  264.3800 Position: 3
2019-03-14, ADBE   Order:  55	Type: Buy  	Status Completed 	Size:    3.0000 Price:  264.3800 Position: 3
2019-03-14, ADBE   BUY   Price: 265.26 Cost: 795.78 Comm: 0.00 Size:    3.0000 
2019-03-18, Sell 3 shares of stock at 257.76
2019-03-19, ADBE   Order:  56	Type: Sell 	Status Accepted 	Size:   -3.0000 Price:  257.7600 Position: 0
2019-03-19, ADBE   Order:  56	Type: Sell 	Status Completed 	Size:   -3.0000 Price:  257.7600 Position: 0
2019-03-19, ADBE   SELL  Price: 259.43 Cost: 795.78 Comm: 0.00 Size:   -3.0000 
2019-03-19, ADBE Closed: PnL Gross -17.49, Net -17.5,
2019-03-22, Buy 3 shares of stock at 259.69
2019-03-25, ADBE   Order:  57	Type: Buy  	Status Accepted 	Size:    3.0000 Price:  259.6900 Position: 3
2019-03-25, ADBE   Order:  57	Type: Buy  	Status Completed 	Size:    3.0000 Pric

<IPython.core.display.Javascript object>

starting portfolio value: 1000.00
position size: 0
2019-01-25, Buy 20 shares of stock at 48.76
2019-01-28, SONY   Order: 100	Type: Buy  	Status Accepted 	Size:   20.0000 Price:   48.7600 Position: 20
2019-01-28, SONY   Order: 100	Type: Buy  	Status Completed 	Size:   20.0000 Price:   48.7600 Position: 20
2019-01-28, SONY   BUY   Price:  48.52 Cost: 970.40 Comm: 0.00 Size:   20.0000 
2019-01-28, Sell 20 shares of stock at 48.83
2019-01-29, SONY   Order: 101	Type: Sell 	Status Accepted 	Size:  -20.0000 Price:   48.8300 Position: 0
2019-01-29, SONY   Order: 101	Type: Sell 	Status Completed 	Size:  -20.0000 Price:   48.8300 Position: 0
2019-01-29, SONY   SELL  Price:  49.04 Cost: 970.40 Comm: 0.00 Size:  -20.0000 
2019-01-29, SONY Closed: PnL Gross 10.4, Net 10.4,
2019-01-31, Buy 19 shares of stock at 50.12
2019-02-01, SONY   Order: 102	Type: Buy  	Status Accepted 	Size:   19.0000 Price:   50.1200 Position: 19
2019-02-01, SONY   Order: 102	Type: Buy  	Status Completed 	Size:   19.0000 Pric

<IPython.core.display.Javascript object>

Solving environment: done

# All requested packages already installed.

